In [1]:
%load_ext autoreload
%autoreload 2

# Import utils

In [2]:
import sympy as sp
import numpy as np

# Load directory ../Packages/PySW/ to path
import sys
sys.path.append('../Packages/PySW/')

from IPython.display import display, Math, Markdown

In [3]:
from Modules.sympy.classes import *
from Modules.sympy.utils import *
from Modules.sympy.untruncated.solver import *

In [4]:
import matplotlib.pyplot as plt

# Latex Plot Beautification
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=18)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
plt.rc('ytick', labelsize=16)    # fontsize of the tick labels
plt.rc('legend', fontsize=16)    # legend fontsize

# Transparent background for figures but keep the legend background white but its text color black
plt.rcParams['legend.facecolor'] = 'white'
plt.rcParams['legend.edgecolor'] = 'none'
plt.rcParams['legend.framealpha'] = 0.8
plt.rcParams['legend.shadow'] = False
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['axes.facecolor'] = 'none'
# White spins, axis and ticks 
plt.rcParams['text.color'] = 'black'
plt.rcParams['axes.labelcolor'] = 'white'
plt.rcParams['xtick.color'] = 'white'
plt.rcParams['ytick.color'] = 'white'
plt.rcParams['axes.labelcolor'] = 'white'
plt.rcParams['axes.edgecolor'] = 'white'


In [5]:
class HObject:
    def __init__(self, H):
        self.H = H
        eigvals, eigvecs = np.linalg.eig(H)
        indx_sorted_eigvals = np.argsort(eigvals)
        
        self.eigvals = eigvals[indx_sorted_eigvals]
        self.eigvecs = [eigvecs[:, arg] for arg in indx_sorted_eigvals]

        self.projectors = [state.reshape(-1, 1).dot(state.reshape(-1, 1).T.conj()) for state in self.eigvecs]

        self.grond_state = eigvecs[0]

        self.initial_state = np.copy(self.grond_state)

    # latex representation of the object
    def _repr_latex_(self):
        # Whole description, hamiltonian, eigenvalues, eigenvectors, projectors and ground state
        latex_str = r'\begin{align*}'
        latex_str += r'\text{Hamiltonian} & : ' + sp.latex(sp.Matrix(self.H)) + r'\\'
        latex_str += r'\text{Eigenvalues} & : ' + sp.latex(sp.Matrix(self.eigvals)) + r'\\'
        latex_str += r'\text{Eigenvectors} & : ' + sp.latex(sp.Matrix(self.eigvecs)) + r'\\'
        latex_str += r'\text{Projectors} & : ' + r'\\'
        for projector in self.projectors:
            latex_str += sp.latex(sp.Matrix(projector)) + r'\\'

        latex_str += r'\text{Ground State} & : ' + sp.latex(sp.Matrix(self.grond_state)) + r'\\'
        latex_str += r'\end{align*}'
        return latex_str

    def compute_projectors(self, state):
        result_projectors = []
        for projector in self.projectors:
            result_projectors.append(state.T.conj().dot(projector).dot(state))
        return result_projectors

# Hamiltonian

In [6]:
dim_bosonic = 3
dim_spin = 2

Spin = RDBasis("\\sigma", 'spin', dim = dim_spin)
s0, sx, sy, sz = Spin._basis

a_1 = RDBoson("a_1", subspace ="boson1", dim_projection=dim_bosonic)
ad_1 = RDBoson("{a_1^\\dagger}", subspace ="boson1", is_annihilation=False, dim_projection=dim_bosonic)
a_2 = RDBoson("a_2", subspace ="boson2", dim_projection=dim_bosonic)
ad_2 = RDBoson("{a_2^\\dagger}", subspace ="boson2", is_annihilation=False, dim_projection=dim_bosonic)

commutation_relations = {
    a_1 * ad_1 : ad_1 * a_1 + 1,
    a_2 * ad_2 : ad_2 * a_2 + 1,
    a_2 * ad_1 : ad_1 * a_2,
    ad_2 * a_1 : a_1 * ad_2,
    ad_2 * ad_1 : ad_1 * ad_2,
    a_2 * a_1 : a_1 * a_2
}

subs_spaces = [[Spin.subspace, dim_spin], [a_1.subspace, dim_bosonic], [a_2.subspace, dim_bosonic]]

# Effective length of the y-axis : l_y = l_x * sqrt(k_y)
ky = RDsymbol("k_y", order = 0)

# zero-th order symbols
omega_plus = RDsymbol("omega_+", order = 0)
omega_minus = RDsymbol("omega_-", order = 0)
omega_z = RDsymbol("omega_z", order = 0)

H0 = omega_plus * ad_1 * a_1 + omega_minus * ad_2 * a_2 - sp.Rational(1, 2) * omega_z * sz
display_dict({Operator('H_0'): H0})

# first order symbols
g_plus = RDsymbol("g_+", order = 1)
g_minus = RDsymbol("g_-", order = 1)

V1 = sp.Rational(1, 2) * (g_plus * (ad_1 + a_1) * sx + sp.I * g_minus * (ad_2 - a_2) * sx + g_plus * sp.sqrt(ky) * (ad_2 + a_2) * sy + sp.I * g_minus * sp.sqrt(ky) * (ad_1 - a_1) * sy)
display_dict({Operator('V_1'): V1})

# second order symbols
Omega_plus = RDsymbol("Omega_+", order = 2)
Omega_minus = RDsymbol("Omega_-", order = 2)
Omega_z = RDsymbol("Omega_z", order = 2)

varphi_1 = RDsymbol("\\varphi_1", order = 0)
varphi_2 = RDsymbol("\\varphi_2", order = 0)
Phi_1 = RDsymbol("\\Phi_1", order = 2)
phi_1 = RDsymbol("\\phi_1", order = 0)
Phi_2 = RDsymbol("\\Phi_2", order = 2)
phi_2 = RDsymbol("\\phi_2", order = 0)

H2 = sp.Rational(1,2) * Omega_plus * ad_1*a_1*sz + sp.Rational(1,2) * Omega_minus * ad_2*a_2*sz + sp.Rational(1,4) * Omega_z * sz
display_dict({Operator('H_2'): H2})
V2 = sp.Rational(1, 4) * (Omega_plus * sp.exp(sp.I* varphi_1) * a_1**2 * sz + Omega_minus * sp.exp(sp.I* varphi_2) * a_2**2 * sz + Phi_1 * sp.exp(sp.I* phi_1) * a_1*a_2 * sz + Phi_2 * sp.exp(sp.I* phi_2) * a_1* ad_2 * sz)
V2 += sp.Rational(1, 4) * (Omega_plus * sp.exp(-sp.I* varphi_1) * ad_1**2 * sz + Omega_minus * sp.exp(-sp.I* varphi_2) * ad_2**2 * sz + Phi_1 * sp.exp(-sp.I* phi_1) * ad_1*ad_2 * sz + Phi_2 * sp.exp(-sp.I* phi_2) * ad_1* a_2 * sz)
display_dict({Operator('V_2'): V2})

# Final Hamiltonian
H = (H0 + V1 + V2 + H2).expand()
display_dict({Operator('H'): H})

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Simplification dicts

In [7]:
n1, n2 = sp.symbols('n_1 n_2', real=True, positive=True)
B = sp.symbols('B', real=True)
t = n2/B
K_plus, K_minus = sp.symbols('K_+ K_-', real=True, positive=True)

kB = sp.symbols('k_B', real=True, positive=True)
phi = 2 * ky**2 / (kB * sp.sqrt(K_plus * K_minus * ky) * (ky**2 - 1))
theta = sp.symbols('\\theta', real=True)

Tz = K_plus * (1 + ky) * sp.cos(theta)**2 + K_minus * (1 + ky) * sp.sin(theta)**2

simplification_dict = {
    g_plus : n1 / (B * sp.sqrt(2)) * sp.sqrt(K_plus) * sp.cos(theta),
    g_minus : n1 / (B * sp.sqrt(2)) * sp.sqrt(K_minus) * sp.sin(theta),
    Omega_z : t * Tz * omega_z
}


# Untruncated solution

In [8]:
sol = solver(H, composite_basis=Spin, order=2, full_diagonal=True, commutation_relations=commutation_relations)

100%|██████████| 6/6 [00:02<00:00,  2.03it/s]


In [9]:
H_2_dict = group_by_list_operators(sol[0], [a_1, ad_1, a_2, ad_2, sx, sy, sz])
display_dict(H_2_dict)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [10]:
omega_qubit = -H_2_dict[sz]*2
a,b = sp.symbols('a b', real=True, positive=True)
phi_symbol = a/b
omega_qubit = sp.factor_terms(sp.trigsimp(sp.factor_terms(omega_qubit).subs(simplification_dict)).expand()).collect([sp.cos(theta), sp.sin(theta), sp.sin(2*theta)])
display(omega_qubit)

omega_z + (-sqrt(k_y)*omega_+*sqrt(K_+)*sqrt(K_-)*n_1**2/(4*B**2*(omega_+**2 - omega_z**2)) + sqrt(k_y)*omega_-*sqrt(K_+)*sqrt(K_-)*n_1**2/(4*B**2*(omega_-**2 - omega_z**2)))*sin(2*\theta) + (-k_y*omega_z*K_+*n_2/(2*B) - k_y*omega_z*K_+*n_1**2/(4*B**2*(omega_-**2 - omega_z**2)) - omega_z*K_+*n_2/(2*B) - omega_z*K_+*n_1**2/(4*B**2*(omega_+**2 - omega_z**2)))*cos(\theta)**2 + (-k_y*omega_z*K_-*n_2/(2*B) - k_y*omega_z*K_-*n_1**2/(4*B**2*(omega_+**2 - omega_z**2)) - omega_z*K_-*n_2/(2*B) - omega_z*K_-*n_1**2/(4*B**2*(omega_-**2 - omega_z**2)))*sin(\theta)**2

In [11]:
k3 = sp.symbols('k_3', real=True, positive=True)
kp, km = sp.symbols('k_+ k_-', real=True, positive=True)
kz = sp.symbols('k_z', real=True, positive=True)

subs_omegas = {
    omega_plus : 1/kp,
    omega_minus : 1/km,
    omega_z : 1/kz,
}

subs_trigs = {
    sp.cos(theta)**2 : (sp.cos(theta)**2).subs({theta: sp.atan(-phi_symbol)/2}).trigsimp().simplify(),
    sp.sin(theta)**2 : (sp.sin(theta)**2).subs({theta: sp.atan(-phi_symbol)/2}).trigsimp().simplify(),
    sp.sin(2*theta) : (sp.sin(2*theta)).subs({theta: sp.atan(-phi_symbol)/2}).trigsimp().simplify(),
    sp.cos(2*theta) : (sp.cos(2*theta)).subs({theta: sp.atan(-phi_symbol)/2}).trigsimp().simplify(),
}

omega_qubit.subs(subs_trigs)

omega_z - a*(-sqrt(k_y)*omega_+*sqrt(K_+)*sqrt(K_-)*n_1**2/(4*B**2*(omega_+**2 - omega_z**2)) + sqrt(k_y)*omega_-*sqrt(K_+)*sqrt(K_-)*n_1**2/(4*B**2*(omega_-**2 - omega_z**2)))/sqrt(a**2 + b**2) + (-b/(2*sqrt(a**2 + b**2)) + 1/2)*(-k_y*omega_z*K_-*n_2/(2*B) - k_y*omega_z*K_-*n_1**2/(4*B**2*(omega_+**2 - omega_z**2)) - omega_z*K_-*n_2/(2*B) - omega_z*K_-*n_1**2/(4*B**2*(omega_-**2 - omega_z**2))) + (b/(2*sqrt(a**2 + b**2)) + 1/2)*(-k_y*omega_z*K_+*n_2/(2*B) - k_y*omega_z*K_+*n_1**2/(4*B**2*(omega_-**2 - omega_z**2)) - omega_z*K_+*n_2/(2*B) - omega_z*K_+*n_1**2/(4*B**2*(omega_+**2 - omega_z**2)))

In [12]:
# Term 1
subs_sqrt_a2_b2 = {
    sp.sqrt(a**2 + b**2) : (1/kp**2 - 1/km**2)
}
gamma = sp.symbols('\\gamma', real=True, positive=True)
subs_kz = {
    kz : kB / gamma
}
sp.factor_terms(omega_qubit.subs(subs_trigs).as_ordered_terms()[1].subs({sp.sqrt(K_plus*K_minus*ky) : 2/(a * kB)})).subs(subs_sqrt_a2_b2).subs(subs_omegas).factor().expand().subs({kp*km:ky}).factor().subs({kp*km:ky})

k_y**2*k_z**2*n_1**2*(k_y + k_z**2)/(2*B**2*k_B*(k_+ + k_-)*(-k_y**2 + k_+**2*k_z**2 + k_-**2*k_z**2 - k_z**4))

In [13]:
# Term 2

term12 = ky / (omega_minus**2 - omega_z**2) + 1/(omega_plus**2 - omega_z**2)
term12.subs(subs_omegas).cancel().factor().subs({ky:kp*km}).factor().subs({kp*km:ky}).factor().collect([ky])

k_+*k_z**2*(-k_y**2*k_- - k_y*k_- + k_+*k_z**2 + k_-**3*k_z**2)/((k_+ - k_z)*(k_+ + k_z)*(k_- - k_z)*(k_- + k_z))

In [14]:
n1, n2 = sp.symbols('n_1 n_2', real=True, positive=True)

subs_Ks_plus_Ks = {
    K_plus + K_minus : 2*(ky+1)*(kp+km) / (ky**2 + kp**2 + km**2 +1),
    K_plus - K_minus : -2*(ky-1)*(kp-km) / (ky**2 + kp**2 + km**2 +1)
}

b_sqrt_a2_b2 = (ky**2 -1) / (km**2 - kp**2)
T1 = -n2/(4*B) * (K_plus + K_minus) / kz * (1 + ky)
T2 = -n2/(4*B) * (K_plus - K_minus) / kz * (1 + ky) * b_sqrt_a2_b2
(T1 + T2).subs(subs_Ks_plus_Ks).factor().subs({kp*km:ky})

-n_2*(k_y + 1)**2/(2*B*k_z*(k_+ + k_-))

In [15]:
subs_Ks = {
    K_plus : 2*kp / (kp**2 + 1),
    K_minus : 2*km / (km**2 + 1)
}

subs_Ks_plus_Ks.update(
    {
        ky*K_plus  - K_minus: (ky*K_plus  - K_minus).subs(subs_Ks).cancel().subs({kp*km:ky}).factor(),
        K_plus - ky*K_minus : (K_plus - ky*K_minus).subs(subs_Ks).cancel().subs({kp*km:ky}).factor(),
        ky*K_plus + K_minus : (ky*K_plus + K_minus).subs(subs_Ks).cancel().subs({kp*km:ky}).factor(),
        K_plus + ky*K_minus : (K_plus + ky*K_minus).subs(subs_Ks).cancel().subs({kp*km:ky}).factor()
    }
)

display_dict(subs_Ks_plus_Ks)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [16]:
T1 = (km**2 * (kp**2 - kz**2) * (ky*K_plus + K_minus))
T2 = (km**2 * (kp**2 - kz**2) * (ky*K_plus - K_minus))

T3 = kp**2 * (km**2 - kz**2) * (ky*K_minus + K_plus)
T4 = kp**2 * (km**2 - kz**2) * (K_plus  - ky*K_minus)

T11 = ((T1 + T2 * b_sqrt_a2_b2)).subs(subs_Ks_plus_Ks).factor().subs({kp*km:ky}).factor().subs({ky:kp*km}).factor()
display(T11)
T22 = ((T3 + T4 * b_sqrt_a2_b2)).subs(subs_Ks_plus_Ks).factor().subs({kp*km:ky}).factor().subs({ky:kp*km}).factor()
display(T22)
sp.factor_terms(((T11 + T22) / ((km**2 - kz**2) * (kp**2 - kz**2))))

-2*k_-**3*(k_+ - k_z)*(k_+ + k_z)*(k_+**2*k_-**2 - 2*k_+**2 + 1)/((k_+ - k_-)*(k_+ + k_-))

2*k_+**3*(k_- - k_z)*(k_- + k_z)*(k_+**2*k_-**2 - 2*k_-**2 + 1)/((k_+ - k_-)*(k_+ + k_-))

2*(k_+**3*(k_- - k_z)*(k_- + k_z)*(k_+**2*k_-**2 - 2*k_-**2 + 1) - k_-**3*(k_+ - k_z)*(k_+ + k_z)*(k_+**2*k_-**2 - 2*k_+**2 + 1))/((k_+ - k_-)*(k_+ + k_-)*(k_+**2 - k_z**2)*(k_-**2 - k_z**2))

## Driving Term - Lab Frame

In [17]:
# Driving in unities of hbar omega_x
E0 = sp.symbols('E_0', real=True, positive=True)
H_Driving =  - E0 * (g_plus * (ad_1 + a_1) + sp.I * g_minus * (ad_2 - a_2))
display_dict({Operator('H_{\\text{Driving}}'): H_Driving})

<IPython.core.display.Math object>

In [18]:
H_driving_rotatetd = rotate_with_S(H_Driving, sol[1], order=1, commutation_relations=commutation_relations, composite_basis=Spin)

100%|██████████| 3/3 [00:00<00:00,  9.17it/s]


In [19]:
subs_sqrt_a2_b2

{sqrt(a**2 + b**2): -1/k_-**2 + k_+**(-2)}

In [20]:
H_drive_terms = H_driving_rotatetd[1].factor().subs(simplification_dict).trigsimp().collect([sp.cos(2*theta), sp.sin(2*theta)]).subs(subs_trigs).subs({sp.sqrt(K_plus*K_minus*ky) : 2/(a * kB)}).collect([K_plus, K_minus]).as_ordered_factors()
first_part = Mul(*H_drive_terms[:-2]).subs(subs_omegas).factor().subs({kp*km:ky})
display(first_part)
second_part = H_drive_terms[-2].expand().subs({b/sp.sqrt(a**2+b**2):b_sqrt_a2_b2}).subs(subs_sqrt_a2_b2).expand().subs(subs_omegas).expand().collect([K_plus,K_minus]).subs(subs_Ks).cancel().subs({kp*km:ky}).factor().expand()
second_part

k_y**2*E_0*k_z**4*n_1**2/(4*B**2*(k_+ - k_z)*(k_+ + k_z)*(k_- - k_z)*(k_- + k_z))

-2/k_z**2 + 2/(k_B*k_z) + 2/k_y**2

In [21]:
sp.factor_terms(first_part * second_part).factor().collect(kB)

-E_0*k_z**2*n_1**2*(-k_y**2*k_z + k_B*(k_y**2 - k_z**2))/(2*B**2*k_B*(k_+ - k_z)*(k_+ + k_z)*(k_- - k_z)*(k_- + k_z))

# Driving Term Co-moving Frame